# 01. Market Expectations In The Cross-Section Of Present Values

TBD

In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from settings import config
from pathlib import Path

import pull_ken_french_data
import pull_fred

# Load environment variables
DATA_DIR = Path(config("DATA_DIR"))
OUTPUT_DIR = Path(config("OUTPUT_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")
START_DATE = config("START_DATE")
END_DATE = config("END_DATE")

## Prep Data

### Step 1: Fama-French Data

In [ ]:
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web

get_available_datasets()

In [ ]:
bm_6_portfolios = web.DataReader("6_Portfolios_2x3", "famafrench")

In [ ]:
print(bm_6_portfolios['DESCR'])

In [ ]:
bm_6_portfolios[6].head()

In [ ]:
# This will pull the needed portfolio data from Ken French's website
# Running doit will pull the data automatically and save it to the data directory

bm_6_portfolios = pull_ken_french_data.pull_ken_french_excel(
    dataset_name="6_Portfolios_2x3",
    data_dir=DATA_DIR,
    log=True,
    start_date=START_DATE,
    end_date=END_DATE,
)

bm_25_portfolios = pull_ken_french_data.pull_ken_french_excel(
    dataset_name="25_Portfolios_5x5",
    data_dir=DATA_DIR,
    log=True,
    start_date=START_DATE,
    end_date=END_DATE,
)

bm_100_portfolios = pull_ken_french_data.pull_ken_french_excel(
    dataset_name="100_Portfolios_10x10",
    data_dir=DATA_DIR,
    log=True,
    start_date=START_DATE,
    end_date=END_DATE,
)

### Step 2: CRSP Market Capitalization Weighted Index Data

### Step 3: Risk-Free Rate Data

In [ ]:
rf = pull_fred.load_fred()
rf

In [ ]:
rf = rf['TB3MS']
rf = rf[(rf.index >= START_DATE) & (rf.index <= END_DATE)]

In [ ]:
rf_monthly = rf.dropna()

In [ ]:
rf_monthly